1.1 수학과 파이썬 복습

In [15]:
import numpy as np

x = np.array([1,2,3])
print(x.__class__)
print(x.shape)
print(x.ndim)

<class 'numpy.ndarray'>
(3,)
1


In [16]:
W = np.array([[1,2,3], [4,5,6]])
print(W.shape)
print(W.ndim)

(2, 3)
2


In [17]:
W = np.array([[1,2,3], [4,5,6]])
X = np.array([[0,1,2], [3,4,5]])
W + X

array([[ 1,  3,  5],
       [ 7,  9, 11]])

In [18]:
W * X

array([[ 0,  2,  6],
       [12, 20, 30]])

In [19]:
A = np.array([[1,2], [3,4]])
A * 10

array([[10, 20],
       [30, 40]])

In [20]:
b = np.array([10,20])
A * b

array([[10, 40],
       [30, 80]])

In [21]:
a = np.array([1,2,3])
b = np.array([4,5,6])
np.dot(a,b)

32

In [22]:
A = np.array([[1,2], [3,4]])
B = np.array([[5,6], [7,8]])
np.matmul(A,B)

array([[19, 22],
       [43, 50]])

1.2 신경망 추론

In [23]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [24]:
a = sigmoid(h)

In [25]:
import numpy as np

W1 = np.random.randn(2,4)
b1 = np.random.randn(4)
W2 = np.random.randn(4,3)
b2 = np.random.randn(3)
x = np.random.randn(10,2)
h = np.matmul(x, W1) + b1
s = np.matmul(a, W2) + b2

In [26]:
class Sigmoid:
    def __init__(self):
        self.params = []
        
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

In [27]:
class Affine:
    def __init__(self,W,b):
        self.params = [W,b]
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x,W) + b
        return out

In [30]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        W1 = np.random.randn(I,H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H,O)
        b2 = np.random.randn(O)
        
        self.layers = [
            Affine(W1,b1),
            Sigmoid(),
            Affine(W2,b2)
        ]
        
        self.params = []
        for layer in self.layers:
            self.params += layer.params
    
    def predict(self,x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [31]:
x = np.random.randn(10,2)
model = TwoLayerNet(2,4,3)
s = model.predict(x)

1.3.1 손실 함수

MatMul 노드

In [33]:
class Matmul:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = x
        return out
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x,W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T,dout)
        self.grads[0][...] = dW
        return dx

Affine 계층

In [34]:
class Affine:
    def __init__(self,W,b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x,W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T,dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

Softmax with Loss 계층

In [35]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        loss = cross_entropy_error(self.y, self.t)
        return loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size
        return dx

Sigmoid 계층

In [36]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [37]:
class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y =  1 / (1 + np.exp(-x))
        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) * dout / batch_size
        return dx

In [38]:
class Dropout:
    def __init__(self, dropout_ratio = 0.5):
        self.params, self.grads = [], []
        self.dropout_ratio = dropout_ratio
        self.mask = None
        
    def forward(self, x, train_flg = True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)
    
    def backward(self, dout):
        return dout * self.mask

In [39]:
class Embedding:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, x):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None

Repeat 노드

In [5]:
import numpy as np
D,N = 8,7
x = np.random.randn(1,D)
y = np.repeat(x,N,axis=0)
dy = np.random.randn(N,D)
dx = np.sum(dy, axis=0, keepdims=True)

Sum 노드

In [7]:
D,N = 8,7
x = np.random.randn(N,D)
y = np.sum(x,axis=0, keepdims=True)
dy = np.random.randn(1,D)
dx = np.repeat(dy, N, axis=0)

1.3.6 가중치 갱신

In [8]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr #= learning rate
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]